1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [11]:
# If haven't downloaded it yet, please get the data file with wget
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

--2022-11-23 13:24:35--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Risoluzione di www.dropbox.com (www.dropbox.com)... 162.125.69.18
Connessione a www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 302 Found
Posizione: /s/raw/xvjzaxzz3ysphme/data_000637.txt [segue]
--2022-11-23 13:24:35--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Riutilizzo della connessione esistente a www.dropbox.com:443.
Richiesta HTTP inviata, in attesa di risposta... 302 Found
Posizione: https://ucf450284d8c4c7e66fe908da213.dl.dropboxusercontent.com/cd/0/inline/BxSuCBbAt4MUesgLELPpXobZFh06OvO3rEltw-s7dCccNyVrTGk_L5t6BWiKrOuOWtO1d9otDn-qAyC1In5-dah8driCiceuAMcRpQQAN0ypqgR9LiDhhrPtkxrWIKmtUhlxph8PW37p6-HomDznfx-ZmgbqdT5-nb3ggmV9pujVsg/file# [segue]
--2022-11-23 13:24:36--  https://ucf450284d8c4c7e66fe908da213.dl.dropboxusercontent.com/cd/0/inline/BxSuCBbAt4MUesgLELPpXobZFh06OvO3rEltw-s7dCccNyVrTGk_L5t6BWiKrOuOWtO1d9o

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [2]:
import numpy as np
import pandas as pd
import datetime as dt


dataf = pd.read_csv('data/data_000637.txt.2', nrows = np.random.randint(1000,1310721))
dataf

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
149118,1,0,29,3869201651,929,26
149119,1,0,43,3869201651,1987,28
149120,1,0,46,3869201651,1992,20
149121,1,0,123,3869201651,1995,21


2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [3]:
x = dataf.groupby('ORBIT_CNT')['BX_COUNTER'].transform(np.max).unique().mean()
x

2922.982972136223

3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

In [10]:
dataf['Absolute Time (ns)'] = (dataf['TDC_MEAS']*(25/30) + dataf['BX_COUNTER']*25).cumsum()
dataf['Absolute Time (Datetime)'] = pd.to_datetime(dataf['Absolute Time (ns)'])
dataf

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,Absolute Time (ns),Absolute Time (Datetime)
0,1,0,123,3869200167,2374,26,5.937167e+04,1970-01-01 00:00:00.000059371
1,1,0,124,3869200167,2374,27,1.187442e+05,1970-01-01 00:00:00.000118744
2,1,0,63,3869200167,2553,28,1.825925e+05,1970-01-01 00:00:00.000182592
3,1,0,64,3869200167,2558,19,2.465583e+05,1970-01-01 00:00:00.000246558
4,1,0,64,3869200167,2760,25,3.155792e+05,1970-01-01 00:00:00.000315579
...,...,...,...,...,...,...,...,...
149118,1,0,29,3869201651,929,26,6.671518e+09,1970-01-01 00:00:06.671518380
149119,1,0,43,3869201651,1987,28,6.671568e+09,1970-01-01 00:00:06.671568079
149120,1,0,46,3869201651,1992,20,6.671618e+09,1970-01-01 00:00:06.671617895
149121,1,0,123,3869201651,1995,21,6.671668e+09,1970-01-01 00:00:06.671667788


4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [44]:
duration = dataf.iloc[-1,7] - dataf.iloc[0,7]
print(duration)
(duration_h, duration_m, duration_s) = (((dataf.iloc[-1,7] - dataf.iloc[0,7]).seconds)/3600, ((dataf.iloc[-1,7] - dataf.iloc[0,7]).seconds)/60, ((dataf.iloc[-1,7] - dataf.iloc[0,7]).seconds))


(duration_h1, duration_m1, duration_sec1) = ((dataf.iloc[-1,7].value - dataf.iloc[0,7].value)/((10**9)*3600), (dataf.iloc[-1,7].value - dataf.loc[0,'Absolute Time (Datetime)'].value)/((10**9)*60), (dataf.iloc[-1,7].value - dataf.loc[0,'Absolute Time (Datetime)'].value)/(10**9)) 


print(duration_h, duration_m, duration_sec)
print(duration_h1, duration_m1, duration_sec1)

0 days 00:00:06.671658367
0.0016666666666666668 0.1 6671.658367
0.0018532384352777777 0.11119430611666667 6.671658367


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [62]:
#counts = dataf.groupby(['FPGA','TDC_CHANNEL'])['TDC_CHANNEL'].count().sort_values(ascending = False) #è una serie
#print(counts.head(3))
print(dataf.groupby(['FPGA','TDC_CHANNEL'])['TDC_CHANNEL'].count().sort_values(ascending = False).head(3))

FPGA  TDC_CHANNEL
0     139            8704
      64             7240
      63             7163
Name: TDC_CHANNEL, dtype: int64


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [91]:
(dataf.groupby('ORBIT_CNT')['ORBIT_CNT'].count() >= 1).count()

1483

7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [92]:
dataf[dataf['TDC_CHANNEL'] == 139]['ORBIT_CNT'].unique().shape[0]

1481

8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [122]:
FPGA0 = (dataf[dataf['FPGA']== 0]).groupby('TDC_CHANNEL')['TDC_CHANNEL'].count()
FPGA1 = (dataf[dataf['FPGA']==1]).groupby('TDC_CHANNEL')['TDC_CHANNEL'].count()
print(FPGA0,'\n')
print(FPGA1,'\n')

TDC_CHANNEL
1       139
2       170
3       173
4       240
5       177
       ... 
129       1
130       5
137       4
138       4
139    8704
Name: TDC_CHANNEL, Length: 122, dtype: int64 

TDC_CHANNEL
1      3206
2      3685
3      2493
4      2988
5      1699
       ... 
129       4
130       6
137       4
138       4
139    3702
Name: TDC_CHANNEL, Length: 132, dtype: int64 



9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.